In [13]:
# Import required libraries:
import sqlite3
import pandas as pd

In [15]:
# Create a new SQLite database (will be saved as a file on your PC)
conn = sqlite3.connect("Chinook_Sqlite.sqlite")
conn.close()

In [16]:
# Connect to the new Chinook database
conn = sqlite3.connect("Chinook_Sqlite.sqlite")

# Example: load Album.csv into the Album table
df = pd.read_csv("Album.csv")
df.to_sql("Album", conn, if_exists="replace", index=False)

# Repeat for all other CSV files
tables = ["Artist","Customer","Employee","Genre","Invoice","InvoiceLine",
          "MediaType","Playlist","PlaylistTrack","Track"]

for table in tables:
    df = pd.read_csv(f"{table}.csv")
    df.to_sql(table, conn, if_exists="replace", index=False)

# Check if tables are created
check = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(check)

             name
0           Album
1          Artist
2        Customer
3        Employee
4           Genre
5         Invoice
6     InvoiceLine
7       MediaType
8        Playlist
9   PlaylistTrack
10          Track




---



**Top-Selling Products**

In [17]:
query = """
SELECT t.Name AS TrackName, SUM(il.Quantity) AS TotalSold
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
GROUP BY t.TrackId
ORDER BY TotalSold DESC
LIMIT 10;
"""
top_tracks = pd.read_sql_query(query, conn)
print("Top 10 Selling Tracks:")
display(top_tracks)

Top 10 Selling Tracks:


,TrackName,TotalSold
0,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",2
1,"Music for the Funeral of Queen Mary: VI. ""Thou...",2
2,"Suite No. 3 in D, BWV 1068: III. Gavotte I & II",2
3,Rehab,2
4,"Symphonie Fantastique, Op. 14: V. Songe d'une ...",2
5,"Scheherazade, Op. 35: I. The Sea and Sindbad's...",2
6,War Pigs,2
7,Beautiful Boy,2
8,Give Peace a Chance,2
9,Pilot,2


**Revenue per Region**

In [18]:
query = """
SELECT c.Country, SUM(i.Total) AS Revenue
FROM Invoice i
JOIN Customer c ON i.CustomerId = c.CustomerId
GROUP BY c.Country
ORDER BY Revenue DESC;
"""
revenue_region = pd.read_sql_query(query, conn)
print("Revenue per Country:")
display(revenue_region)


Revenue per Country:


,Country,Revenue
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
5,United Kingdom,112.86
6,Czech Republic,90.24
7,Portugal,77.24
8,India,75.26
9,Chile,46.62


**Monthly Performance**

In [19]:
query = """
SELECT strftime('%Y-%m', InvoiceDate) AS Month, SUM(Total) AS Revenue
FROM Invoice
GROUP BY Month
ORDER BY Month;
"""
monthly_revenue = pd.read_sql_query(query, conn)
print("Monthly Revenue Trend:")
display(monthly_revenue)

Monthly Revenue Trend:


,Month,Revenue
0,2009-01,35.64
1,2009-02,37.62
2,2009-03,37.62
3,2009-04,37.62
4,2009-05,37.62
5,2009-06,37.62
6,2009-07,37.62
7,2009-08,37.62
8,2009-09,37.62
9,2009-10,37.62


**Top Artists by Sales**

In [20]:
query = """
SELECT ar.Name AS Artist, SUM(il.Quantity) AS TotalSold
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
JOIN Album al ON t.AlbumId = al.AlbumId
JOIN Artist ar ON al.ArtistId = ar.ArtistId
GROUP BY ar.ArtistId
ORDER BY TotalSold DESC
LIMIT 10;
"""
top_artists = pd.read_sql_query(query, conn)
print("Top 10 Artists by Sales:")
display(top_artists)

Top 10 Artists by Sales:


,Artist,TotalSold
0,Iron Maiden,140
1,U2,107
2,Metallica,91
3,Led Zeppelin,87
4,Os Paralamas Do Sucesso,45
5,Deep Purple,44
6,Faith No More,42
7,Lost,41
8,Eric Clapton,40
9,R.E.M.,39


**Top Customers per Country (Using RANK)**

In [22]:
query = """
SELECT
    c.Country,
    c.FirstName || ' ' || c.LastName AS Customer,
    SUM(i.Total) AS Revenue,
    RANK() OVER(PARTITION BY c.Country ORDER BY SUM(i.Total) DESC) AS RankInCountry
FROM Invoice i
JOIN Customer c ON i.CustomerId = c.CustomerId
GROUP BY c.CustomerId
ORDER BY c.Country, RankInCountry;
"""
top_customers = pd.read_sql_query(query, conn)
print("Top Customers per Country (with RANK):")
display(top_customers.head(20))  # show first 20 rows

Top Customers per Country (with RANK):


,Country,Customer,Revenue,RankInCountry
0,Argentina,Diego Gutiérrez,37.62,1
1,Australia,Mark Taylor,37.62,1
2,Austria,Astrid Gruber,42.62,1
3,Belgium,Daan Peeters,37.62,1
4,Brazil,Luís Gonçalves,39.62,1
5,Brazil,Eduardo Martins,37.62,2
6,Brazil,Alexandre Rocha,37.62,2
7,Brazil,Fernanda Ramos,37.62,2
8,Brazil,Roberto Almeida,37.62,5
9,Canada,François Tremblay,39.62,1
